In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
plotly.offline.init_notebook_mode(connected=True)

In [3]:
def ham(N, xperiodicity, yperiodicity, u):
    """Calculates the QWZ Hamiltonian of the 2D lattice."""
    
    matrix1 = diag(ones(N))
    matrix2 = roll(matrix1,1,axis=1)
    matrix2_open = copy(matrix2)
    matrix2_open[-1,0] = 0
    
    #first term in H
    if yperiodicity == 1:
        mat1 = kron(matrix2,matrix1)
    else:
        mat1 = kron(matrix2_open,matrix1)
        
    struc1 = 1/2 * array([[1., 1.j], [1.j, -1]])
    
    #second term in H
    if xperiodicity == 1:
        mat2 = kron(matrix1,matrix2)
    else:
        mat2 = kron(matrix1,matrix2_open)
    
    struc2 = 1/2 * array([[1., 1.], [-1., -1.]])
    
    #third term in H
    mat3 = diag(ones(N**2))
    
    struc3 = u * array([[1, 0], [0, -1]])   
    
    #Hamiltonian
    H = kron(mat1, struc1) + kron(mat2, struc2)
    H += H.T.conj()
    H += kron(mat3, struc3)
    
    return H

In [4]:
def disturbham(H0, weight):
    """Adds disorder to the Hamiltonian."""
    disturbstruc = array([[1, 0],[0, 1]])
    disturbance = kron(weight * diag(rand(N**2)-0.5), disturbstruc)
    
    return H0 + disturbance

In [5]:
"""This code calculates the eigenstates multiple disorder realizations for
the same disorder strength (x-axis) with different random numbers, the
plot shows the expected behavior, the eigenstates get mixed up after a
certain disorder strength."""

N = 3
yperiodicity = 1 #0 to switch it off, otherwise it is periodic
xperiodicity = 1 
u = 3.
max_weight = 7

simnum = 30
simulationrange = linspace(0,1,simnum)

xvals = []

H0 = ham(N, xperiodicity, yperiodicity, u)

figure = {
    'data': [],
    'layout': {'title': 'Weight = 0'},
    'frames': []
}

frames_dic = {'frame' + str(x) : [] for x in range(max_weight)}
step = 0.5

for weight in arange(0, max_weight+step, step):
    
    deltaes = []
    energies = []

    for simulations in simulationrange:

        H = disturbham(H0, weight)
        ee, vv = eigh(H)

        deltae = delete(ee - roll(ee,1), 0)
        deltaes.append(deltae)

        energies.append(ee)

    frame_vals = []
    
    if weight == 0:
        for jj in range(1, simnum+1):
            xvals.append(repeat(jj/simnum, len(ee)))

        flat_xvals = [item for sublist in xvals for item in sublist]
    
    flat_energies = [item for sublist in energies for item in sublist]
    
    trace = go.Scatter(
    x = flat_xvals,
    y = flat_energies,
    mode = 'markers',
    marker= dict(size= 4,
                 color = 'blue'
                   ))

    if weight == 0:
        figure['data'].append(trace)
    
    else:
        figure['frames'].append({'data': [trace], 'layout' :{'title': 'Weight = ' + str(weight)}})               

maximum = max(array(energies).flatten()) * 1.1
figure['layout']['yaxis'] = {'title': 'Energy spectrum',
                            'range' : [maximum,-maximum]}
figure['layout']['xaxis'] = {'showticklabels': False}
        
iplot(figure)